In [1]:
!pip install -q tensorflow

import numpy as np
import tensorflow as tf
import random as rn

In [2]:
import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(37)
rn.seed(1254)
tf.random.set_seed(89)

In [3]:
from tensorflow.python.keras import backend as K

# force tensorflow to use a single thread
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),config=config)
K.set_session(sess)

In [4]:
## Imports libs
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

from tensorflow.keras import layers
from keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

Segmentation Models: using `tf.keras` framework.


Using TensorFlow backend.


In [5]:
!pip install -U -q segmentation-models
!pip install -q keras
!pip install -q tensorflow-estimator

import pandas as pd
import cv2
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
from random import shuffle
from tqdm import tqdm
import pandas as pd
from sklearn.utils import class_weight

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
#from tensorflow.keras.utils.generic_utils import CustomObjectScope
from tensorflow.keras.layers import Flatten, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from tensorflow.keras.metrics import top_k_categorical_accuracy
from sklearn.metrics import confusion_matrix

In [6]:
BATCH_SIZE=80
INPUT_SIZE = (224, 224)

In [7]:
model=load_model(r'C:\Users\HP\FER_SL\models\affectnet_emotions\mobilenet_7.h5')
model.summary()
net_description='mobilenet_7'
def mobilenet_preprocess_input(x,**kwargs):
    x[..., 0] -= 103.939
    x[..., 1] -= 116.779
    x[..., 2] -= 123.68
    return x
preprocessing_function=mobilenet_preprocess_input
        
BATCH_SIZE=80
INPUT_SIZE = (224, 224)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [8]:
net_description='sl_'+net_description

In [9]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function,horizontal_flip = True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

In [10]:
target_size = (224, 224)

training_set = train_datagen.flow_from_directory(r'C:\Users\HP\FER_SL\sl_data\train_files_cropped', target_size=target_size,
                                                 batch_size=BATCH_SIZE, class_mode='categorical',follow_links=True)

test_set = test_datagen.flow_from_directory(r'C:\Users\HP\FER_SL\sl_data\test_files_cropped', target_size=target_size,
                                            batch_size=BATCH_SIZE, class_mode='categorical',follow_links=True)

Found 2024 images belonging to 7 classes.
Found 507 images belonging to 7 classes.


# Class weights for imbalanced classes

In [11]:
(unique, counts) = np.unique(training_set.classes, return_counts=True)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(class_weights)

{0: 1.7331536388140163, 1: 4.121794871794871, 2: 2.5617529880478087, 3: 4.01875, 4: 4.2026143790849675, 5: 2.217241379310345, 6: 1.0}


# Downloading the previous model

In [12]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [13]:
def count_params(model):
    non_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.non_trainable_weights])
    trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_weights])
    return {'non_trainable_params': non_trainable_params, 'trainable_params': trainable_params}

In [14]:
params = count_params(model)
assert params['non_trainable_params'] == 21888
assert params['trainable_params'] == 3471175

# Removing the last layer

In [15]:
new_model = Sequential()
for layer in model.layers[:-1]:
    new_model.add(layer)

# Freezing the previous layers

In [16]:
for layer in new_model.layers:
    layer.trainable = False

# Adding a new dense layer

In [17]:
print(model.input)

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [18]:
new_model.add(Dense(units=7, input_shape=(224,224), activation='softmax'))
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0

# Training only the last layer

In [19]:
params = count_params(new_model)
assert params['non_trainable_params'] == 3491264
# only the parameters in the last layer are trainable
assert params['trainable_params'] == 1799

In [20]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(net_description)
nb_train_samples = 2024
nb_validation_samples = 507

mc = ModelCheckpoint(net_description+'.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
es=EarlyStopping(monitor='val_accuracy',patience=2)
FIRST_EPOCHS=3

hist1=new_model.fit(training_set, steps_per_epoch=nb_train_samples//BATCH_SIZE, epochs=FIRST_EPOCHS, verbose=1, 
                    initial_epoch=0, callbacks=[mc, es], validation_data=test_set, validation_steps=nb_validation_samples // BATCH_SIZE,class_weight=class_weights)

sl_mobilenet_7
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/3
25/25 [==============================] - 59s 2s/step - loss: 4.8665 - accuracy: 0.0988 - val_loss: 2.0380 - val_accuracy: 0.1396

Epoch 00001: val_accuracy improved from -inf to 0.13958, saving model to sl_mobilenet_7.h5
Epoch 2/3
25/25 [==============================] - 54s 2s/step - loss: 3.9375 - accuracy: 0.2906 - val_loss: 1.7149 - val_accuracy: 0.3292

Epoch 00002: val_accuracy improved from 0.13958 to 0.32917, saving model to sl_mobilenet_7.h5
Epoch 3/3
25/25 [==============================] - 54s 2s/step - loss: 3.6111 - accuracy: 0.3699 - val_loss: 1.5549 - val_accuracy: 0.4604

Epoch 00003: val_accuracy improved from 0.32917 to 0.46042, saving model to sl_mobilenet_7.h5


# Re-training the model with all the layers

In [21]:
start_epoch=len(hist1.history['loss'])

for layer in new_model.layers:
    layer.trainable = True
    
new_model.summary()

params = count_params(new_model)
assert params['non_trainable_params'] == 21888
# only the parameters in the last layer are trainable
assert params['trainable_params'] == 3471175 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0

In [22]:
new_model.load_weights(net_description+'.h5')
new_model.compile(optimizer=Adam(learning_rate=1e-4,decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
SECOND_EPOCHS=start_epoch+5

mc = ModelCheckpoint(net_description+'_ft.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
#es=EarlyStopping(monitor='val_accuracy',patience=2 )

print('All tuning')
hist2=new_model.fit(training_set, steps_per_epoch=training_set.samples//BATCH_SIZE, epochs=SECOND_EPOCHS, verbose=1, 
                    initial_epoch=start_epoch, validation_data=test_set, validation_steps=test_set.samples // BATCH_SIZE, callbacks=[mc],class_weight=class_weights)

All tuning
Epoch 4/8
25/25 [==============================] - 262s 10s/step - loss: 3.2967 - accuracy: 0.4470 - val_loss: 1.3666 - val_accuracy: 0.5354

Epoch 00004: val_accuracy improved from -inf to 0.53542, saving model to sl_mobilenet_7_ft.h5
Epoch 5/8
25/25 [==============================] - 250s 10s/step - loss: 2.6024 - accuracy: 0.5730 - val_loss: 1.2264 - val_accuracy: 0.5729

Epoch 00005: val_accuracy improved from 0.53542 to 0.57292, saving model to sl_mobilenet_7_ft.h5
Epoch 6/8
25/25 [==============================] - 247s 10s/step - loss: 2.0894 - accuracy: 0.6445 - val_loss: 1.0770 - val_accuracy: 0.6146

Epoch 00006: val_accuracy improved from 0.57292 to 0.61458, saving model to sl_mobilenet_7_ft.h5
Epoch 7/8
25/25 [==============================] - 254s 10s/step - loss: 1.7421 - accuracy: 0.6940 - val_loss: 0.9962 - val_accuracy: 0.6146

Epoch 00007: val_accuracy did not improve from 0.61458
Epoch 8/8
25/25 [==============================] - 252s 10s/step - loss: 1.475

In [24]:
new_model.load_weights(net_description+'_ft.h5')

In [25]:
new_model.compile(optimizer=SGD(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
#new_model.summary()
mc = ModelCheckpoint(net_description+'_ft_sgd.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
second_epoch=start_epoch+len(hist2.history['loss'])
THIRD_EPOCHS=2+second_epoch

print('ALL SGD tuning')
hist3=new_model.fit_generator(training_set, steps_per_epoch=training_set.samples//BATCH_SIZE, epochs=THIRD_EPOCHS, verbose=1, 
                    initial_epoch=second_epoch, validation_data=test_set, validation_steps=test_set.samples // BATCH_SIZE,callbacks=[mc],class_weight=class_weights)

ALL SGD tuning


c:\program files\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 9/10
25/25 [==============================] - 255s 10s/step - loss: 1.2914 - accuracy: 0.7747 - val_loss: 0.9850 - val_accuracy: 0.6292

Epoch 00009: val_accuracy improved from -inf to 0.62917, saving model to sl_mobilenet_7_ft_sgd.h5
Epoch 10/10
25/25 [==============================] - 251s 10s/step - loss: 1.3218 - accuracy: 0.7721 - val_loss: 0.9724 - val_accuracy: 0.6354

Epoch 00010: val_accuracy improved from 0.62917 to 0.63542, saving model to sl_mobilenet_7_ft_sgd.h5


In [26]:
USE_PREPROCESSING=True
print(INPUT_SIZE)

(224, 224)


In [27]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

# Evaluation

In [1]:
import numpy as np
import cv2
y_val=np.array(test_set.labels)
y_scores_val=[]
#new_model.load_weights(net_description+'_ft.h5')
for filepath in log_progress(test_set.filepaths):
    img=cv2.imread(filepath)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,INPUT_SIZE)
    if USE_PREPROCESSING:
        inp = preprocessing_function(np.expand_dims(img, axis=0).astype(np.float32))
    else:
        inp=img.astype(np.float32)
        inp[..., 0] -= 103.939
        inp[..., 1] -= 116.779
        inp[..., 2] -= 123.68
        inp = np.expand_dims(inp, axis=0)

    scores=new_model.predict(inp)
    #print(scores)
    y_scores_val.append(scores)

y_scores_val=np.concatenate(y_scores_val, axis=0)
#print(y_scores_val.shape,y_val.shape)

NameError: name 'test_set' is not defined

In [29]:
idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}

y_pred=np.argmax(y_scores_val,axis=1)
acc=100.0*(y_val==y_pred).sum()/len(y_val)
print('Accuracy:',acc)

y_train=np.array(training_set.labels)

for i in range(y_scores_val.shape[1]):
    _val_acc=(y_pred[y_val==i]==i).sum()/(y_val==i).sum()
    print('%s %d/%d acc: %f' %(idx_to_class[i],(y_train==i).sum(),(y_val==i).sum(),100*_val_acc))

Accuracy: 63.70808678500986
Anger 371/132 acc: 75.000000
Disgust 156/31 acc: 74.193548
Fear 251/57 acc: 56.140351
Happiness 160/19 acc: 84.210526
Neutral 153/42 acc: 38.095238
Sadness 290/59 acc: 52.542373
Surprise 643/167 acc: 63.473054


In [30]:
from sklearn.metrics import classification_report

target_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', 'Surprise']
print(classification_report(y_val, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Anger       0.76      0.75      0.75       132
     Disgust       0.66      0.74      0.70        31
        Fear       0.43      0.56      0.49        57
       Happy       0.44      0.84      0.58        19
     Neutral       0.39      0.38      0.39        42
     Sadness       0.48      0.53      0.50        59
    Surprise       0.85      0.63      0.73       167

    accuracy                           0.64       507
   macro avg       0.57      0.63      0.59       507
weighted avg       0.67      0.64      0.64       507



# Making a single prediction

In [2]:
from facial_analysis import FacialImageProcessing
imgProcessing=FacialImageProcessing(False)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential, load_model,model_from_json

my_model = load_model(r"C:\Users\HP\SLR\1src\sl_mobilenet_7_ft_sgd.h5") 
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
#mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = face_mesh.process(image)
        
        img = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #cv2.imshow('MediaPipe FaceMesh', img)
        
        INPUT_SIZE = (224, 224)
        idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}
        #filename = r'C:\Users\HP\SLR\test_files_1_sorted\07January_2010_Thursday_tagesschau_default-5.avi_pid0_fn000045-0.png'

        #img = plt.imread(filename)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img=cv2.resize(img,INPUT_SIZE)
        #frame_bgr=cv2.imread(filename)
        #frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(frame)
        bounding_boxes, points = imgProcessing.detect_faces(frame)
        points = points.T
        for bbox,p in zip(bounding_boxes, points):
            box = bbox.astype(np.int)
            x1,y1,x2,y2=box[0:4]    
            face_img=frame[y1:y2,x1:x2,:]

            face_img=cv2.resize(face_img,INPUT_SIZE)

            #plt.imshow(face_img)

            inp=face_img.astype(np.float32)
            inp[..., 0] -= 103.939
            inp[..., 1] -= 116.779
            inp[..., 2] -= 123.68
            inp = np.expand_dims(inp, axis=0)
            scores=my_model.predict(inp)
            #plt.title(idx_to_class[np.argmax(scores)])
            
            cv2.putText(frame, idx_to_class[np.argmax(scores)], (0, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 192, 203), 2)
            
            cv2.imshow('video', frame)
            
            #print(idx_to_class[np.argmax(scores)])

            #print(scores)

        
        if cv2.waitKey(5) & 0xFF == 27:
              break

cap.release()

In [ ]:
# For image 02August_2010_Monday_heute_default-7.avi_pid0_fn000008-0.png

INPUT_SIZE = (224, 224)
idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}
#filename = r'C:\Users\HP\SLR\test_files_1_sorted\07January_2010_Thursday_tagesschau_default-5.avi_pid0_fn000045-0.png'

#img = plt.imread(filename)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img=cv2.resize(img,INPUT_SIZE)
#frame_bgr=cv2.imread(filename)
frame_bgr=cv2.imread(img)
frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
plt.imshow(frame)
bounding_boxes, points = imgProcessing.detect_faces(frame)
points = points.T
for bbox,p in zip(bounding_boxes, points):
    box = bbox.astype(np.int)
    x1,y1,x2,y2=box[0:4]    
    face_img=frame[y1:y2,x1:x2,:]

    face_img=cv2.resize(face_img,INPUT_SIZE)

    plt.imshow(face_img)

    inp=face_img.astype(np.float32)
    inp[..., 0] -= 103.939
    inp[..., 1] -= 116.779
    inp[..., 2] -= 123.68
    inp = np.expand_dims(inp, axis=0)
    scores=new_model.predict(inp)
    plt.title(idx_to_class[np.argmax(scores)])

    print(scores)